In [1]:
import glob
import pandas as pd
from datetime import datetime
import os
import contextlib
import shutil
from geopy.geocoders import Nominatim
import numpy as np
import pyodbc
import warnings

warnings.filterwarnings('ignore')

In [2]:
#Paste link for file

df = pd.read_csv('https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/e86f8e38-a23c-4c1a-8455-c8f94210a8f1/download/crime-incident-reports-2018.csv')
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I192077559,3115,Investigate Person,INVESTIGATE PERSON,B3,468,NaN,2018-04-30 09:00:00,2018,4,Monday,9,Part Three,HAZLETON ST,42.279971,-71.095534,"(42.27997063, -71.09553354)"
1,I192077332,619,Larceny,LARCENY ALL OTHERS,E18,496,NaN,2018-03-06 08:00:00,2018,3,Tuesday,8,Part One,HYDE PARK AVE,42.269224,-71.120853,"(42.26922388, -71.12085347)"
2,I192076660,2629,Harassment,HARASSMENT,E5,662,NaN,2018-10-31 12:00:00,2018,10,Wednesday,12,Part Two,PRIMROSE ST,42.290765,-71.130211,"(42.29076521, -71.13021098)"
3,I192075386,2629,Harassment,HARASSMENT,A1,96,NaN,2018-04-09 08:43:00,2018,4,Monday,8,Part Two,ATLANTIC AVE,42.355264,-71.050988,"(42.35526402, -71.05098788)"
4,I192075335,3208,Property Lost,PROPERTY - MISSING,D4,132,NaN,2018-01-01 00:00:00,2018,1,Monday,0,Part Three,COMMONWEALTH AVE,42.353522,-71.072838,"(42.35352153, -71.07283786)"


In [3]:
# check for ETL_Folder
etl = "Folder_path/ETL_Folder"

if not os.path.exists(etl):
    os.makedirs(etl)

In [4]:
# timestamp_format = '%Y%m%d_%H%M'
timestamp_format = "%Y%m%d_%H%M"

#Year-Month-Date_Hour_Minute

now = datetime.now() # get current timestamp

timestamp = now.strftime(timestamp_format)

parent_dir = etl

directory = "/BostonCrime_Data_" + str(timestamp) #Change the name of the data

path = parent_dir + directory

#Creating a folder based on timestamp
if not os.path.exists(path):
    os.makedirs(path)
    
# Directory within to keep original data
original_data = path + "/Original_Data"

# Creating that directory
if not os.path.exists(original_data):
    os.makedirs(original_data)

# sending the csv file to the created directory
df.to_csv(path_or_buf = original_data + '/crime-incident-reports-Boston.csv', index = False)

In [5]:
# logfile = path + "/" + "bostonCrime_logfile.txt"            # all event logs will be stored
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(path + "/" + "bostonCrime_logfile.txt","a") as f: f.write("\n" + timestamp + ',' + message + 'n')

-----------------------------------------------------

Inserting Original Table in SQL Server

Note: A copy of the original file has been created in the ETL_Folder

In [6]:
log("ETL Job Started")
log("Storing Original Table in SQL Server")

In [7]:
server = 'Enter Server Name'
database = 'Enter Database Name'

#establish connection string
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                      SERVER='+ server +'; \
                      DATABASE='+ database +';\
                      Trusted_Connection=yes;')
cursor = conn.cursor()

In [8]:
#checks if Table exists
cursor.execute(''' if not exists (select * from sysobjects where name='Original_Table' and xtype='U')
Create Table Original_Table( INCIDENT_NUMBER varchar(500),
                                                OFFENSE_CODE int ,
                                                OFFENSE_CODE_GROUP varchar(500),
                                                OFFENSE_DESCRIPTION varchar(500),
                                                DISTRICT varchar(500) , 
                                                REPORTING_AREA varchar(500) , 
                                                SHOOTING varchar(500),
                                                OCCURRED_ON_DATE varchar(500), 
                                                YEAR int, 
                                                MONTH int, 
                                                DAY_OF_WEEK varchar(500), 
                                                HOUR varchar(500), 
                                                UCR_PART varchar(500),
                                                STREET varchar(500), 
                                                Lat float, 
                                                Long float , 
                                                Location varchar(500) )''')
conn.commit()

In [9]:
#nsert query
insert_query_og = "INSERT INTO [ETL_Database].[dbo].[Original_Table](INCIDENT_NUMBER, OFFENSE_CODE, OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION, DISTRICT, REPORTING_AREA, SHOOTING,OCCURRED_ON_DATE, YEAR, MONTH, DAY_OF_WEEK, HOUR, UCR_PART,STREET, Lat, Long, Location) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

In [10]:
#converting Python nan to SQL NULL

#storing data in Server
for index, row in df.replace({np.NAN: None}).iterrows():
     cursor.execute(insert_query_og, row.INCIDENT_NUMBER, row.OFFENSE_CODE, row.OFFENSE_CODE_GROUP,row.OFFENSE_DESCRIPTION, row.DISTRICT, row.REPORTING_AREA, row.SHOOTING,row.OCCURRED_ON_DATE, row.YEAR, row.MONTH, row.DAY_OF_WEEK, row.HOUR, row.UCR_PART,row.STREET, row.Lat, row.Long, row.Location)

conn.commit()
cursor.close()
conn.close()

In [11]:
log("Original Data has been stored to SQL Server")

----------------------------------------------------

Extracting the data into a folder

In [12]:
log("Extraction process has started")

In [13]:
#Reads and returns the dataframe
def extract_csv(file_name): 
    dataframe = pd.read_csv(file_name, on_bad_lines = 'warn') 
    return dataframe 

In [14]:
# extract function
def extract():
    df = pd.DataFrame(columns = ['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP','OFFENSE_DESCRIPTION', 
                                 'DISTRICT', 'REPORTING_AREA', 'SHOOTING','OCCURRED_ON_DATE', 'YEAR', 'MONTH',
                                 'DAY_OF_WEEK', 'HOUR', 'UCR_PART','STREET', 'Lat', 'Long', 'Location'])
    
    for file in glob.glob(original_data + "/*.csv"):
        extracted_data = df.append(extract_csv(file),ignore_index=True)
        
    #extract bad lines and stores them in another text file, in the same folder as the existing file.
    with open(path+'/error_lines.txt', 'w') as log:
        with contextlib.redirect_stderr(log):
            for file in glob.glob(path + "/*.csv"):
                dataframe = extract_csv(file)
        
    return extracted_data

In [15]:
extracted_data = extract()

In [16]:
# storing the extracted data into another folder
extract_data = path + "/Extracted_Data"

if not os.path.exists(extract_data):
    os.makedirs(extract_data)

extracted_data.to_csv(path_or_buf = extract_data + '/extracted-crime-incident-reports.csv', index = False)

In [17]:
log("Extraction process has ended")

--------------------------------------------------------------------------------------

Transforming the data to be used for model development (or atleast that is the intention)

In [18]:
log("Transformation process has started")

In [19]:
#transform function

#maintaining control number 
def transform(df):
    for i in range(len(df['Location'])): #converting Location varibale to list
        df['Location'][i] = list(map(float,df['Location'][i].strip('()').split(','))) 
    
    df['SHOOTING'] = df['SHOOTING'].fillna(0) #replacing null values with 0
    
    df['SHOOTING'] = df['SHOOTING'].replace(to_replace = 'Y', value = 1) #replacing yes values with 1
    
    for i, value in df[pd.isnull(df).any(axis=1)].iterrows(): # gets approximate location using Nominatim
        if(pd.notna(value['STREET'])):
            address= value['STREET'] + ' , Boston, United States'
            geolocator = Nominatim(user_agent="ETL")
            location = geolocator.geocode(address)
            if(location==None):
                continue
            df.loc[i ,'Lat'] = location.latitude
            df.loc[i ,'Long'] = location.longitude
            df['Location'][i] = [location.latitude, location.longitude]
            
        if(pd.isna(value['DISTRICT'])): #replacing null values with empty string
            df.loc[i, 'DISTRICT'] = ' '
            
    #sees if the location is available        
    df['loc_available'] = np.where((pd.isna(df['Lat']) | pd.isna(df['Long']) | pd.isna(df['STREET'] )),0,1)
    df['district_available'] = np.where(df['DISTRICT']==' ',0,1)
    df['UCR_Part_available'] = np.where(pd.isna(df['UCR_PART']),0,1)

    
    #converting string to integer
    df[['OFFENSE_CODE', 'YEAR', 'MONTH', 'HOUR']] = df[['OFFENSE_CODE', 'YEAR', 'MONTH', 'HOUR']].astype(int)
    
    
    return df

In [20]:
transformed_data = transform(extracted_data)

error_data = transformed_data[pd.isnull(transformed_data).any(axis=1)]
# transformed_data.dropna(inplace = True)

In [22]:
# storing the transformed data into another folder
transform_data = path + "/Transformed_Data"

if not os.path.exists(transform_data):
    os.makedirs(transform_data)

transformed_data.to_csv(path_or_buf = transform_data + '/transformed-crime-incident-reports.csv', index = False)
error_data.to_csv(path_or_buf = transform_data + '/error-data-crime-incident-reports.csv', index = False)

In [23]:
log("Transformation process has ended")

-----------------------------

In [24]:
transformed_data = pd.read_csv(transform_data + '/transformed-crime-incident-reports.csv')

In [25]:
transformed_data.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,loc_available,district_available,UCR_Part_available
0,I192077559,3115,Investigate Person,INVESTIGATE PERSON,B3,468,0,2018-04-30 09:00:00,2018,4,Monday,9,Part Three,HAZLETON ST,42.279971,-71.095534,"[42.27997063, -71.09553354]",1,1,1
1,I192077332,619,Larceny,LARCENY ALL OTHERS,E18,496,0,2018-03-06 08:00:00,2018,3,Tuesday,8,Part One,HYDE PARK AVE,42.269224,-71.120853,"[42.26922388, -71.12085347]",1,1,1
2,I192076660,2629,Harassment,HARASSMENT,E5,662,0,2018-10-31 12:00:00,2018,10,Wednesday,12,Part Two,PRIMROSE ST,42.290765,-71.130211,"[42.29076521, -71.13021098]",1,1,1
3,I192075386,2629,Harassment,HARASSMENT,A1,96,0,2018-04-09 08:43:00,2018,4,Monday,8,Part Two,ATLANTIC AVE,42.355264,-71.050988,"[42.35526402, -71.05098788]",1,1,1
4,I192075335,3208,Property Lost,PROPERTY - MISSING,D4,132,0,2018-01-01 00:00:00,2018,1,Monday,0,Part Three,COMMONWEALTH AVE,42.353522,-71.072838,"[42.35352153, -71.07283786]",1,1,1


Loading transformed data into SQL

In [26]:
log("Storing transformed data into SQL Server")

In [27]:
server = 'Enter Server Name'
database = 'Enter Database Name'

#establish connection string
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                      SERVER='+ server +'; \
                      DATABASE='+ database +';\
                      Trusted_Connection=yes;')
cursor = conn.cursor()

In [28]:
#check if table exists
cursor.execute(''' if not exists (select * from sysobjects where name='Transformed_Table' and xtype='U')
Create Table Transformed_Table( INCIDENT_NUMBER varchar(500),
                                                OFFENSE_CODE int ,
                                                OFFENSE_CODE_GROUP varchar(500),
                                                OFFENSE_DESCRIPTION varchar(500),
                                                DISTRICT varchar(500) , 
                                                REPORTING_AREA varchar(500) , 
                                                SHOOTING int,
                                                OCCURRED_ON_DATE DATE, 
                                                YEAR int, 
                                                MONTH int, 
                                                DAY_OF_WEEK varchar(500), 
                                                HOUR int, 
                                                UCR_PART varchar(500),
                                                STREET varchar(500), 
                                                Lat float, 
                                                Long float , 
                                                Location varchar(500),
                                                loc_available int,
                                                district_available int,
                                                UCR_Part_available int)''')
conn.commit()

In [29]:
insert_query_transformed = "INSERT INTO [ETL_Database].[dbo].[Transformed_Table](INCIDENT_NUMBER, OFFENSE_CODE, OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION, DISTRICT, REPORTING_AREA, SHOOTING,OCCURRED_ON_DATE, YEAR, MONTH, DAY_OF_WEEK, HOUR, UCR_PART,STREET, Lat, Long, Location, loc_available,district_available,UCR_Part_available) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

In [30]:
#converting Python nan to SQL NULL

#storing data into SQL Server
for index, row in transformed_data.replace({np.NAN: None}).iterrows():
     cursor.execute(insert_query_transformed, row.INCIDENT_NUMBER, row.OFFENSE_CODE, row.OFFENSE_CODE_GROUP,row.OFFENSE_DESCRIPTION, row.DISTRICT, row.REPORTING_AREA, row.SHOOTING,row.OCCURRED_ON_DATE, row.YEAR, row.MONTH, row.DAY_OF_WEEK, row.HOUR, row.UCR_PART,row.STREET, row.Lat, row.Long, row.Location, row.loc_available, row.district_available, row.UCR_Part_available)

conn.commit()
cursor.close()
conn.close()

In [31]:
log("Transformed data has been stored in SQL Server")